In [1]:
public class Arg
{
  public static T IsAny<T>() => throw new InvalidOperationException();
  public static T Is<T>(Func<T, bool> matcher) => throw new InvalidOperationException();
}

The below script needs to be able to find the current output cell; this is an easy method to get it.

In [2]:
using System.Reflection;
using System.Linq.Expressions;

public class Visitor : ExpressionVisitor
{
  private static readonly MethodInfo IsMethod = typeof(Arg).GetMethods(BindingFlags.Static | BindingFlags.Public).Single(x => x.Name == "Is");

  public MethodInfo Method { get; private set; }
  public IReadOnlyList<Func<object, bool>> ArgumentValidators { get; private set; }

  protected override Expression VisitMethodCall(MethodCallExpression node)
  {
    Method = node.Method;

    var argumentValidators = new List<Func<object, bool>>();
    foreach (var argument in node.Arguments)
    {
      if (argument.NodeType == ExpressionType.Constant)
      {
        // mock.Verify(x => x.M(10));
        // x => x == 10
        // mock.Verify(x => x.M(Arg.Is<int>(arg ==> arg == 10)));
        argumentValidators.Add(x => x.Equals(((ConstantExpression)argument).Value)); 
        
      }
      else if (argument.NodeType == ExpressionType.Call)
      {
        VisitItExpression(argumentValidators, (MethodCallExpression)argument);
      }
    }

    ArgumentValidators = argumentValidators;
    return node;
  }
  
  private void VisitItExpression(List<Func<object,bool>> argumentValidators, MethodCallExpression argument)
  {
    if (argument.Method.DeclaringType != typeof(Arg))
    {
      throw new InvalidOperationException("Only Arg.Is* is supported");
    }
    else if (argument.Method.IsGenericMethod)
    {
      var genericMethodDefinition = argument.Method.GetGenericMethodDefinition();
      if (genericMethodDefinition == IsMethod)
      {
        argumentValidators.Add(VisitIsMethod(argument));
      }
      // Handle Other Methods...
    }
  }

  private Func<object, bool> VisitIsMethod(MethodCallExpression argument)
  {
    var arg = (LambdaExpression)argument.Arguments.Single(); // We know that Arg.Is has only one argument
    // Func<int, bool> originalFunc = (int arg) => arg >= 10
    // Func<object, bool> result = (object arg) => ((int arg) => arg >= 10)((int)arg);
    return BuildArgumentValidator(arg.Parameters.Single().Type, arg);
  }

  private Func<object, bool> BuildArgumentValidator(Type castToType, Expression validateExpression)
  {
    var arg = Expression.Parameter(typeof(object));
    var casted = Expression.Convert(arg, castToType);
    var body = Expression.Invoke(validateExpression, casted);
    return Expression.Lambda<Func<object, bool>>(body, arg).Compile();
  }
}


In [8]:
public interface IFoo
{
  void M(int n);
}

Expression<Action<IFoo>> exp = obj => obj.M(Arg.Is<int>(x => x >= 10));

In [9]:
var visitor = new Visitor();
visitor.Visit(exp);
visitor.Method

Void M(Int32) Name M DeclaringType Submission#10+IFoo ReflectedType Submission#10+IFoo MemberType Method MetadataToken 100663302 Module ℛ*ebb4819e-dddd-4d5d-9e81-6ecf382eee85#1-10.dll MDStreamVersion 131072 FullyQualifiedName <Unknown> ModuleVersionId 8688297c-77da-4f72-b690-0e3ec27a0594 MetadataToken 1 ScopeName ℛ*ebb4819e-dddd-4d5d-9e81-6ecf382eee85#1-10.dll Name <Unknown> Assembly ℛ*ebb4819e-dddd-4d5d-9e81-6ecf382eee85#1-10, Version=0.0.0.0, Culture=neutral, PublicKeyToken=null CodeBase file:///C:/Program Files/dotnet/shared/Microsoft.NETCore.App/7.0.4/System.Private.CoreLib.dll FullName ℛ*ebb4819e-dddd-4d5d-9e81-6ecf382eee85#1-10, Version=0.0.0.0, Culture=neutral, PublicKeyToken=null EntryPoint <null> DefinedTypes index value 0 Microsoft.CodeAnalysis.EmbeddedAttribute 1 System.Runtime.CompilerServices.RefSafetyRulesAttribute 2 Submission#10 3 Submission#10+IFoo 4 Submission#10+<<Initialize>>d__0 5 Submission#10+<>c IsCollectible False ManifestModule ℛ*ebb4819e-dddd-4d5d-9e81-6ecf382eee85#1-10.dll MDStreamVersion 131072 FullyQualifiedName <Unknown> ModuleVersionId 8688297c-77da-4f72-b690-0e3ec27a0594 MetadataToken 1 ScopeName ℛ*ebb4819e-dddd-4d5d-9e81-6ecf382eee85#1-10.dll Name <Unknown> Assembly ℛ*ebb4819e-dddd-4d5d-9e81-6ecf382eee85#1-10, Version=0.0.0.0, Culture=neutral, PublicKeyToken=null CodeBase file:///C:/Program Files/dotnet/shared/Microsoft.NETCore.App/7.0.4/System.Private.CoreLib.dll FullName ℛ*ebb4819e-dddd-4d5d-9e81-6ecf382eee85#1-10, Version=0.0.0.0, Culture=neutral, PublicKeyToken=null EntryPoint <null> DefinedTypes index value 0 Microsoft.CodeAnalysis.EmbeddedAttribute 1 System.Runtime.CompilerServices.RefSafetyRulesAttribute 2 Submission#10 3 Submission#10+IFoo 4 Submission#10+< >d__0 5 Submission#10+<>c IsCollectible False ManifestModule ℛ*ebb4819e-dddd-4d5d-9e81-6ecf382eee85#1-10.dll MDStreamVersion 131072 FullyQualifiedName <Unknown> ModuleVersionId 8688297c-77da-4f72-b690-0e3ec27a0594 MetadataToken 1 ScopeName ℛ*ebb4819e-dddd-4d5d-9e81-6ecf382eee85#1-10.dll Name <Unknown> Assembly ℛ*ebb4819e-dddd-4d5d-9e81-6ecf382eee85#1-10, Version=0.0.0.0, Culture=neutral, PublicKeyToken=null ModuleHandle System.ModuleHandle CustomAttributes [ [System.Security.UnverifiableCodeAttribute()], [System.Runtime.CompilerServices.RefSafetyRulesAttribute((Int32)11)] ] ReflectionOnly False Location ImageRuntimeVersion v4.0.30319 GlobalAssemblyCache False HostContext 0 IsDynamic False ExportedTypes index value 0 Microsoft.CodeAnalysis.EmbeddedAttribute 1 System.Runtime.CompilerServices.RefSafetyRulesAttribute 2 Submission#10 3 Submission#10+IFoo 4 Submission#10+< >d__0 5 Submission#10+<>c IsFullyTrusted True CustomAttributes index value 0 [System.Runtime.CompilerServices.CompilationRelaxationsAttribute((Int32)8)] 1 [System.Runtime.CompilerServices.RuntimeCompatibilityAttribute(WrapNonExceptionThrows = True)] 2 [System.Diagnostics.DebuggableAttribute((System.Diagnostics.DebuggableAttribute+DebuggingModes)263)] EscapedCodeBase file:///C:/Program%20Files/dotnet/shared/Microsoft.NETCore.App/7.0.4/System.Private.CoreLib.dll Modules index value 0 ℛ*ebb4819e-dddd-4d5d-9e81-6ecf382eee85#1-10.dll SecurityRuleSet None ModuleHandle System.ModuleHandle MDStreamVersion 131072 CustomAttributes index value 0 [System.Security.UnverifiableCodeAttribute()] Constructor Void .ctor() ConstructorArguments [ ] NamedArguments [ ] AttributeType System.Security.UnverifiableCodeAttribute 1 [System.Runtime.CompilerServices.RefSafetyRulesAttribute((Int32)11)] Constructor Void .ctor(Int32) ConstructorArguments [ (Int32)11 ] NamedArguments [ ] AttributeType System.Runtime.CompilerServices.RefSafetyRulesAttribute ReflectionOnly False Location ImageRuntimeVersion v4.0.30319 GlobalAssemblyCache False HostContext 0 IsDynamic False ExportedTypes index value 0 Microsoft.CodeAnalysis.EmbeddedAttribute 1 System.Runtime.CompilerServices.RefSafetyRulesAttribute 2 Submission#10 3 Submission#10+IFoo 4 Submission#10+<<Initialize>>d__0 5 Submission#10+<>c IsFullyTrusted Tr

In [11]:
visitor.ArgumentValidators[0](10)

True

In [12]:
#r "nuget: Castle.Core"
using Castle.DynamicProxy;

Installed Packages Castle.Core, 5.1.1

In [13]:
using System.Collections.Immutable;

public class MockInvocationMatch
{
  private readonly MethodInfo _method;
  private readonly IReadOnlyList<Func<object, bool>> _argumentValidators;

  public MockInvocationMatch(Expression expression)
  {
    var visitor = new Visitor();
    visitor.Visit(expression);
    (_method, _argumentValidators) = (visitor.Method, visitor.ArgumentValidators);
  }
  public bool IsMatch(MockInvocation invocation) => invocation.Method == _method && invocation.Arguments.Zip(_argumentValidators, 
    (argument, validator) => validator(argument)).All(x => x);
}

public class MockInvocation 
{
  public MockInvocation(IInvocation invocation)
  {
    Arguments = invocation.Arguments.ToImmutableArray();
    Method = invocation.Method;
  }
  
  public MethodInfo Method { get; }
  public ImmutableArray<object> Arguments { get; }
}